In [1]:
import os
import time
from time import strftime, gmtime
import datetime
import argparse
import random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10, MNIST, ImageFolder
from torchvision.transforms.transforms import RandomHorizontalFlip
import torchmetrics

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

# from model import ResNet, Bottleneck, BasicBlock
from models import get_model
from dataset import get_dataset
from utils import save_result, make_folder

import warnings
warnings.filterwarnings('ignore')

In [15]:
test_path = r'C:\Users\gjust\Documents\Github\data\dogs-vs-cats\train'
ckpoint_folder = 'train6'
checkpoint_path = r'C:\Users\gjust\Documents\Github\Pytorch-Cookbook\4. Project\classification\result\{}\vgg16_ckpt.pth'.format(ckpoint_folder)

In [16]:
test_transform = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

test_dataset = ImageFolder(root=test_path, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

checkpoint = torch.load(checkpoint_path)
model = get_model('vgg16', 2, pretrained=True)
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [23]:
fnames, gts, preds = [], [], []
model.eval()
model.cuda()
for i, (images, labels) in enumerate(test_loader):
    outputs = model(images.cuda())
    _, predicted = torch.max(outputs.data, 1)
    
    mis_idx = predicted.cpu() != labels
    gts.append(labels[mis_idx])
    preds.append(predicted[mis_idx])
    
    for idx in torch.where(mis_idx)[0]:
        fnames.append(test_loader.dataset.samples[i*32 + idx][0])
        gts.append(labels[idx])
        preds.append(predicted[mis_idx])
        
    
data = {'fname': fnames, 'gt': gts, 'pred':preds}
df = pd.DataFrame(data)
df.to_csv('result.csv')

ValueError: All arrays must be of the same length

In [29]:
i*32 + idx

tensor(25015)

In [30]:
gts

[tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [25]:
len(gts)

782

In [26]:
len(preds)

782

In [27]:
len(fnames)

12416